In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import datetime
import time
import numpy as np
import pandas as pd
#import pyautogui
import random
import requests
import datetime
import re
import random

In [5]:
hreflst=[]
realtradehist=[]
#아파트 id로 연결하기 위해서 데이터 불러오기
apt_id=pd.read_csv("C:\\Users\\SAMSUNG\\Desktop\\the greta\\check aptid.csv",encoding='utf=8');apt_id

,apt_id,apt_name,province,district,dong,type,house_cnt,building_cnt,floor_min,floor_max,parking,date_complete,FAR,BLR,company,heating,student_class,student_teacher,address_place,address_road
0,146482,YH빌리지,서울시,강남구,개포동,오피스텔,4,1,2,2,2,2017-09-08,297,59,(주)세담종합건설,개별난방도시가스,17.7,10.9,서울시 강남구 개포동 1237-8,논현로20길 22
1,119219,개포래미안포레스트,서울시,강남구,개포동,아파트,2296,-1,7,35,3961,2020-09-28,249,21,삼성물산(주),지역난방열병합,17.7,10.9,서울시 강남구 개포동 1282,개포로 264
2,107513,개포상지리츠빌,서울시,강남구,개포동,아파트,18,1,10,10,41,2003-03-28,202,31,상지건영(주),개별난방도시가스,32.7,18.6,서울시 강남구 개포동 176-1,개포로 405
3,8928,개포자이,서울시,강남구,개포동,아파트,212,4,20,22,502,2004-06-17,250,18,LG건설(주),지역난방열병합,16.2,11.0,서울시 강남구 개포동 12-2,개포로109길 69
4,145017,개포자이르네,서울시,강남구,개포동,아파트 분양권,28,1,8,10,38,1000-01-01,199,57,자이에스앤디,개별난방도시가스,21.9,15.5,서울시 강남구 개포동 656-3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9548,17491,청원다미소(주상복합),서울시,중랑구,중화동,아파트,54,1,11,15,65,2004-10-29,305,33,청원종합건설(주),개별난방도시가스,18.4,11.4,서울시 중랑구 중화동 286-22,동일로136길 10
9549,14082,태능에셈빌(주상복합),서울시,중랑구,중화동,아파트,103,1,15,15,207,2005-03-28,384,58,CNC종합건설(주),개별난방도시가스,18.4,11.4,서울시 중랑구 중화동 314-1,동일로129길 35
9550,102174,한솔e,서울시,중랑구,중화동,아파트,35,2,7,7,35,2004-07-30,0,0,"인수종합건설(주),원섭종합건설(주)",개별난방도시가스,18.4,11.4,서울시 중랑구 중화동 19-16,봉화산로27길 30
9551,824,한신,서울시,중랑구,중화동,아파트,1544,9,23,27,1548,1997-10-11,376,20,한신공영,개별난방도시가스,18.3,13.0,서울시 중랑구 중화동 450,동일로 752


In [10]:
def gethref_df_dong(keyword):
    #시/구/동을 입력으로 받아 hreflst에 해당 지역 부동산 데이터 href를 저장
    #keyword="인천시 계양구 목상동"
    global hreflist
    hreflist=[]
    #호가수집시 저장해놓은 변수
    #idx=keyword.rfind(" ")
    #city=keyword[:idx]
    #dong=keyword[idx+1:]
    #newdong = ''.join([i for i in dong if not i.isdigit()])
    url="https://m.land.naver.com/search/result/" + keyword
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(1)
    try:
        driver.find_element(By.CSS_SELECTOR,"#_countContainer > a.btn_option._complex").click()
    except:
        return []
    time.sleep(1)
    
    
    max_iter=int(driver.find_element(By.CSS_SELECTOR,"#_listContainer > div > div.heading_onsale._countTtl > a > h3 > strong").text)
    itemlist=driver.find_element(By.CSS_SELECTOR,"#_listContainer > div > div.article_box.article_box--complex")
    html = driver.page_source
    time.sleep(1)
    soup = BeautifulSoup(html, 'html.parser')
    now=datetime.datetime.now()
    
    #총 단지 수에 비례해 적절한 횟수만큼 스크롤을 내린다.
    while True:
        driver.execute_script("arguments[0].scrollBy(0, 5000)", itemlist)
        if (datetime.datetime.now()-now).seconds>=max_iter/40:
            break

    html=driver.page_source   
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    hrefall=soup.find_all('a',attrs={'direction' : 'complexDetail'}) 
    
    #href 주소 저장하기
    for i in hrefall:
        hreflist.append("https://m.land.naver.com/complex/info/"+i['_ncid']+"?ptpNo=1")
    hreflist=list(set(hreflist))
    print(str(len(hreflist))+" number of url collected")
    return hreflist 
        


In [11]:
gethref_df_dong("서울시 강남구 개포동")

39 number of url collected


['https://m.land.naver.com/complex/info/102377?ptpNo=1',
 'https://m.land.naver.com/complex/info/152745?ptpNo=1',
 'https://m.land.naver.com/complex/info/113016?ptpNo=1',
 'https://m.land.naver.com/complex/info/107513?ptpNo=1',
 'https://m.land.naver.com/complex/info/16228?ptpNo=1',
 'https://m.land.naver.com/complex/info/152528?ptpNo=1',
 'https://m.land.naver.com/complex/info/10329?ptpNo=1',
 'https://m.land.naver.com/complex/info/145017?ptpNo=1',
 'https://m.land.naver.com/complex/info/113151?ptpNo=1',
 'https://m.land.naver.com/complex/info/16229?ptpNo=1',
 'https://m.land.naver.com/complex/info/140057?ptpNo=1',
 'https://m.land.naver.com/complex/info/1327?ptpNo=1',
 'https://m.land.naver.com/complex/info/119219?ptpNo=1',
 'https://m.land.naver.com/complex/info/483?ptpNo=1',
 'https://m.land.naver.com/complex/info/667?ptpNo=1',
 'https://m.land.naver.com/complex/info/102378?ptpNo=1',
 'https://m.land.naver.com/complex/info/11078?ptpNo=1',
 'https://m.land.naver.com/complex/info/149

In [4]:
def get_aptid(loc,apt):
    #아파트 코드 찾아와서 저장하기
    l=loc.split(" ")
    
    #loc와 일치하는 주소가 하나뿐이면 아래의 과정을 시행할 필요가 없으므로 이를 반영하기
    selected_df=apt_id[(apt_id['address_place']==loc)]
    if len(selected_df['apt_id'])==1:
        return selected_df['apt_id'].iloc[0]
    
    #그렇지 않으면 정석대로 시행해야 한다.
    selected_df=apt_id[(apt_id['apt_name']==apt) & (apt_id['province']==l[0]) & (apt_id['district']==l[1]) & (apt_id['dong']==l[2]) & (apt_id['type']=="아파트")]

    if len(selected_df['apt_id'])==1:
        return selected_df['apt_id'].iloc[0]
    elif len(selected_df['apt_id'])>1:
        print("중복되는 apt id 발견")
        return None
    else:
        #대치래미안하이스턴과 같은 표기의 혼재가 있어서 처리해 주었다.(대치래미안하이스턴=래미안대치하이스턴)
        try:
            for k in range(len(apt_id)):
                testapt=apt_id['apt_name'][k]
                if len(apt)!=len(testapt):
                    continue
                breaking=True
                for letter in testapt:
                    if letter not in apt:
                        breaking=False
                        break
                if breaking==True:
                    #print(testapt)
                    return apt_id[apt_id['apt_name']==testapt]['apt_id'].iloc[0]

        except:
            return None

In [5]:
def make_intcost(text): 
    #given data is like "9억만원" or "9억 3백만원"
    if "억만원" in text:
        text=text.replace(" ","")
        text=text.replace("억","")
        text=text.replace("만원","")
        text=text.replace(",","")
        return int(text)*100000000
    elif "만원" in text:
        if text.find("억")!=-1:
            bil=text[:text.find("억")]
            mil=text.replace(bil+"억","").replace("만원","").replace(",","").replace(" ","")
            return int(bil)*100000000+int(mil)*10000
        else:
            mil=text.replace("만원","").replace(",","")
            return int(mil)*10000
    
    else:
        print("cost_change error")
        return text

In [15]:
def make_intcost2(text):
    #월세가 붙어있는 자료일때 사용한다.
    #Data is like "8억/30만원" or "9억5,000/30만원" or "5,000/30만원"
    l=[]
    idx=text.find('/')
    l.append(int(text[idx+1:-2].replace(",","")))
    text=text[:idx]+"만원"
    if text=="만원":
        l.append(0)
    else:
        l.append(make_intcost(text))
    return l

In [16]:
make_intcost2("1,500/1,040만원")

[1040, 15000000]

In [7]:
def makedate(year,month,day):
    #Date 형식으로 저장한다.
    #given data is like "2023","2월","9일"
    a= datetime.date(int(year),int(month[:-1]),int(day[:-1])).strftime("%y/%m/%d")
    return a
    

In [8]:
def purespace(spacestring):
    #80B-1과같은 형태는 80으로 저장한다.
    for i,v in enumerate(spacestring):
        if not v.isnumeric():
            return int(spacestring[:i])
    return int(spacestring)

In [9]:
print(purespace('71㎡'),purespace("71-1㎡"),purespace("71B-1㎡"))

71 71 71


In [193]:
biglst=[]

In [10]:
def getinfo(url):
    global biglst

    #기본정보 페이지에 있는 정보
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    html=driver.page_source   
    soup=BeautifulSoup(html,'html.parser')
    a=soup.find_all('span',attrs={'class' : 'detail_data_item'}) 
    try:
        sede=int(a[0].text.replace("세대","").replace(",",""))
    except:
        #url에 더 이상 접근이 불가능할 경우
        print("page not available")
        return None
    dong=int(a[1].text[2:-1])                                                                       #동수
    addr=soup.find('p',attrs={'class' : 'p_address_place _addr'}).text                              #주소
    aptname=soup.find('strong',attrs={'class' : 'detail_complex_title'}).text                       #아파트이름
    b=soup.find_all('div',attrs={'class' : 'col_half'})
    txt=b[1].text       
    totfloor=txt[txt.rfind("/")+1:txt.rfind("층")]                                                  #총 층수
    aptid=get_aptid(addr,aptname)                                                                   #아파트 번호

    #시세/실거래 페이지에 있는 정보
    try:
        time.sleep(0.2)
        driver.find_element(By.CSS_SELECTOR,"#_price_trend_tab_cd").click()
    except:
        #실거래 정보가 없는 단지
        print("no real trade history")                                                                  
        return None
    time.sleep(0.1)
    i=1
    
    while True:
        #면적별로 수집해야함(면적버튼 누르기)
        try:
            space=driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[5]/div/section/div/div/div[5]/div/div/ul/li[{}]/a'.format(i))
            space.click()
            space=driver.find_element(By.CSS_SELECTOR,'#_ptp_list_price > ul > li:nth-child({}) > a > span._ptp_nm_cd'.format(i)).text
            
        except:  
            break
        html=driver.page_source   
        soup=BeautifulSoup(html,'html.parser')
        
        driver.implicitly_wait(3)
        
        
        time.sleep(1)
        for x in range(1,4):
            if x==1:
                try:
                    driver.find_element(By.XPATH,'//*[@id="_price_trend_content_cd"]/article[2]/div[1]/div/div/div/a[2]').send_keys(Keys.ENTER)
                    tradetype="매매"
                except:
                    break
            elif x==2:
                driver.find_element(By.XPATH,'//*[@id="_price_trend_content_cd"]/article[2]/div[1]/div/div/div/a[3]').send_keys(Keys.ENTER)
                tradetype="전세"
            else:
                driver.find_element(By.XPATH,'//*[@id="_price_trend_content_cd"]/article[2]/div[1]/div/div/div/a[4]').send_keys(Keys.ENTER)
                tradetype="월세"
            
            #더보기 버튼 누르기
            time.sleep(1)
            try:
                driver.find_element(By.CSS_SELECTOR,'#_price_trend_content_cd > article.article_box.article_box--detail.article_box--realprice._real_price > div.item_area > a').click()
            except:
                None

            #스크롤 끝까지 내리기
            #solution...????
            html=driver.page_source   
            soup=BeautifulSoup(html,'html.parser')

            #혹시라도 정보가 없으면 스크롤 내리는데 시간 쓸 필요 X
            infoall=soup.find_all('li',attrs={'class' : re.compile('^info_realprice_item _info_item_rp\_')})
            if len(infoall)==0:
                scroll=False
            else:
                scroll=True

            a=len(driver.find_elements(By.CLASS_NAME,('info_line_inner')))
            while scroll:
                # 맨 아래로 스크롤을 내린다.
                driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
                # 스크롤 사이 페이지 로딩 시간(6에서 11초 사이 랜덤하게)
                time.sleep(random.randint(6,10)+random.uniform(0,1))
                # html=driver.page_source   
                # soup=BeautifulSoup(html,'html.parser')
                b=len(driver.find_elements(By.CLASS_NAME,('info_line_inner')))
                if a==b:
                    break
                else:
                    a=b
                    
            
            # #실거래가 탭에서 얻을수 있는 추가 정보
            html=driver.page_source   
            soup=BeautifulSoup(html,'html.parser')
            infoall=soup.find_all('li',attrs={'class' : re.compile('^info_realprice_item _info_item_rp\_')})

            for infonum in range(len(infoall)):
                year=infoall[infonum].find('h3').text    #실거래 년도   
                month=infoall[infonum].find('h4').text   #실거래 월
                pricetot=infoall[infonum].find_all('span',attrs={'class' : 'txt_price'})
                for numb in range(len(pricetot)):
                    t=pricetot[numb].text
                    subidx=t.find("(")
                    subidx2=t.rfind(",")
                    price=t[:subidx]+"만원"              #실거래가격
                    day=t[subidx+1:subidx2]              #실거래일
                    specificfloor=t[subidx2+1:-2]        #해당층
                    if tradetype=="월세":
                        l=make_intcost2(price)
                        lst=[sede,dong,addr,aptname,aptid,specificfloor,totfloor,makedate(year,month,day),tradetype,space[:-1],purespace(space),l[1],l[0],datetime.datetime.now().strftime("%y/%m/%d")]
                    else:
                        lst=[sede,dong,addr,aptname,aptid,specificfloor,totfloor,makedate(year,month,day),tradetype,space[:-1],purespace(space),make_intcost(price),None,datetime.datetime.now().strftime("%y/%m/%d")]
                    print(lst)
                    biglst.append(lst)
     
        i+=1   
        
    

In [12]:
biglst=[]

In [13]:
len(biglst)

0

In [14]:
location="서울시 강남구"
keyword=["개포동","논현동","대치동","도곡동","삼성동","세곡동","수서동","신사동","압구정동","역삼동","율현동","일원동","자곡동","청담동"]
for i in keyword:
    biglst=[]
    hreflst=gethref_df_dong(location+i)
    for index,j in enumerate(hreflst):
        getinfo(j)
        print(str(index+1)+"th 단지 complete")
    pddf2=pd.DataFrame(biglst)
    pddf2.columns=['총 세대수','동수','주소',"아파트명",'아파트번호','해당층','총 층','실거래일시','매매유형','타입면적','면적','실거래가','월세','수집일시']
    pddf2.to_csv("C:\\Users\\SAMSUNG\\Desktop\\finaldata\\{}.csv".format("naver "+location+" "+i),encoding='euc-kr')

[28, 1, '서울시 강남구 개포동 1283', '개포자이르네', 145017, '5', '10', '22/12/17', '전세', '71', 71, 500000000, None, '23/02/21']
[28, 1, '서울시 강남구 개포동 1283', '개포자이르네', 145017, '3', '10', '23/01/06', '전세', '85', 85, 600000000, None, '23/02/21']
[28, 1, '서울시 강남구 개포동 1283', '개포자이르네', 145017, '3', '10', '23/01/06', '전세', '85', 85, 600000000, None, '23/02/21']
1th 단지 complete
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '3', '8', '21/10/14', '전세', '116', 116, 585000000, None, '23/02/21']
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '3', '8', '21/09/15', '전세', '116', 116, 585000000, None, '23/02/21']
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '3', '8', '19/07/23', '전세', '116', 116, 520000000, None, '23/02/21']
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '6', '8', '19/01/13', '매매', '152B', 152, 1020000000, None, '23/02/21']
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '5', '8', '17/07/08', '매매', '152B', 152, 800000000, None, '23/02/21']
[19, 1, '서울시 강남구 개포동 1242-2', '개포현대6차', 10863, '5', '8', '0

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 8.40MB/s]


[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '1', '22', '21/08/19', '매매', '160', 160, 2540000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '20', '22', '21/06/16', '매매', '160', 160, 2000000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '6', '22', '21/05/19', '매매', '160', 160, 2398500000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '13', '22', '21/01/21', '매매', '160', 160, 2000000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '8', '22', '21/01/13', '매매', '160', 160, 2395000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '7', '22', '20/02/26', '매매', '160', 160, 2060000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '17', '22', '20/01/06', '매매', '160', 160, 2030000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '10', '22', '19/11/23', '매매', '160', 160, 1950000000, None, '23/02/22']
[212, 4, '서울시 강남구 개포동 12-2', '개포자이', 8928, '19', '22', '19/10/19', '매매', '160', 160,

ValueError: invalid literal for int() with base 10: '1,040'

In [248]:
#인터넷 끊겨서 특정 index에서 재수집(biglst의 985번째줄부터 재수집한다.)
# for index,j in enumerate(hreflst[34:]):
#     getinfo(j)
#     print(str(index+1)+"th 단지 complete")
# pddf2=pd.DataFrame(biglst)
# pddf2.columns=['총 세대수','동수','주소',"아파트명",'아파트번호','해당층','총 층','실거래일시','매매유형','타입면적','면적','실거래가','월세','수집일시']
# pddf2.to_csv("C:\\Users\\SAMSUNG\\Desktop\\finaldata\\{}.csv".format("naver "+location+" "+i),encoding='euc-kr')

[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '9', '9', '18/07/22', '매매', '105B', 105, 1750000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '9', '9', '12/08/11', '전세', '105B', 105, 550000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '7', '9', '22/03/30', '매매', '106A', 106, 2650000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '8', '9', '22/01/15', '매매', '106A', 106, 2720000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '2', '9', '21/09/03', '매매', '106A', 106, 2600000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '6', '9', '21/07/13', '매매', '106A', 106, 2450000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '2', '9', '21/05/08', '매매', '106A', 106, 2480000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '5', '9', '21/01/15', '매매', '106A', 106, 2240000000, None, '23/02/21']
[558, 13, '서울시 강남구 개포동 654', '현대2차', 1321, '3', '9', '20/12/09', '매매', '106A', 106, 21300

In [165]:
#special case(세종시)
# location="세종시"
# keyword=["가람동","고운동"]
# for i in keyword:
#     biglst=[]
#     hreflst=gethref_df_dong(location+i)
      #단지 정보가 없는 경우
#     if hreflst==[]:
#         continue
#     for index,j in enumerate(hreflst):
#         getinfo(j)
#         print(str(index+1)+"th 단지 complete")
#     pddf2=pd.DataFrame(biglst)
#     pddf2.columns=['총 세대수','동수','주소',"아파트명",'아파트번호','해당층','총 층','실거래일시','매매유형','타입면적','면적','실거래가','월세','수집일시']
#     pddf2.to_csv("C:\\Users\\SAMSUNG\\Desktop\\finaldata\\{}.csv".format("naver"+location+" "+i),encoding='euc-kr')